In [1]:
import pandas as pd

url = 'https://data.cdc.gov/api/views/95ax-ymtc/rows.csv?accessType=DOWNLOAD'
dodr_data = pd.read_csv(url)
dodr_data.head()


,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,FLAG
0,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,1999,1,All ages,1.1,6.1,NaN
1,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2000,2,All ages,1.1,6.2,NaN
2,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2001,3,All ages,1.1,6.8,NaN
3,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2002,4,All ages,1.1,8.2,NaN
4,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2003,5,All ages,1.1,8.9,NaN


In [2]:
import pandas as pd

dodr_data['UNIT'] = dodr_data['UNIT'].str.strip()
dodr_data['AGE'] = dodr_data['AGE'].str.strip()

dodr_data['rate_type'] = dodr_data['UNIT'].apply(
    lambda x: 'age-adjusted' if 'age-adjusted' in x else 'crude'
)

dodr_data['age_groups'] = dodr_data['AGE'].apply(
    lambda x: 'All' if (pd.isna(x) or str(x).strip() in ['All ages', 'All']) 
    else str(x).replace(' years', '').replace('years', '').strip()
)

dodr_data.to_csv('clean_overdose_final.csv', index=False)

In [3]:
#renaming panels

dodr_data = dodr_data.rename(columns={
    'PANEL': 'DRUG_TYPE',
    'YEAR_NUM': 'YEARS_AFTER_1998',
    'rate_type': 'RATE_TYPE',
    'age_groups': 'AGE_GROUPS'
})

dodr_data.to_csv('clean_overdose_final.csv', index=False)

    

In [4]:
#SEX and RACE/ETHNICITY

def parse_demo(label):
    sex = None
    race_ethnicity = None

    if label == "All persons":
        sex = "All"
        race_ethnicity = "All"
    elif label in ["Male", "Female"]:
        sex = label
        race_ethnicity = "All"
    elif ":" in label:
        parts = label.split(":", 1)
        sex = parts[0].strip()
        race_ethnicity = parts[1].strip()
    else:
        sex = "Unknown"
        race_ethnicity = label
    
    return pd.Series({
        'SEX': sex,
        'RACE_ETHNICITY': race_ethnicity
    })

dodr_data[['SEX', 'RACE_ETHNICITY']] = dodr_data['STUB_LABEL'].apply(parse_demo)

dodr_data.to_csv('overdose_clean_final.csv', index=False)


In [5]:
#getting rid of columns

columns_to_drop = ['UNIT','FLAG']
dodr_data = dodr_data.drop(columns=columns_to_drop)

print("Columns dropped!")
print(f"\nRemaining columns: {dodr_data.columns.tolist()}")
print(f"Total columns: {len(dodr_data.columns)}")

dodr_data.to_csv('overdose_clean_final.csv', index=False)

Columns dropped!

Remaining columns: ['INDICATOR', 'DRUG_TYPE', 'PANEL_NUM', 'UNIT_NUM', 'STUB_NAME', 'STUB_NAME_NUM', 'STUB_LABEL', 'STUB_LABEL_NUM', 'YEAR', 'YEARS_AFTER_1998', 'AGE', 'AGE_NUM', 'ESTIMATE', 'SEX', 'RACE_ETHNICITY']
Total columns: 15


In [6]:
clean_overdose = pd.read_csv('clean_overdose_final.csv')

if 'PANEL' in clean_overdose.columns:
    clean_overdose = clean_overdose.rename(columns={'PANEL': 'drug_type'})

clean_overdose.to_csv('clean_overdose_final.csv', index=False)